In [1]:
%reload_ext autoreload
%autoreload 2
import colabexts
from colabexts.jcommon import *

jpath=os.path.dirname(colabexts.__file__)
jcom = f'{jpath}/jcommon.ipynb'
%run $jcom

import os, sys, datetime, re, json, importlib
from collections import defaultdict
from sys import modules
from IPython.display import HTML, Javascript
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_rows', 16)


In [2]:
%%writefile /opt/utils/getraw.py
#!/usr/local/bin/python 

import os, sys
import tika
tika.TikaClientOnly = True
from tika import parser
import base64
import textract

#-----------------------------------------------------------------------------------
def getDataTika(ffile):
    data = parser.from_file(ffile)
    text = data['content'].strip() if 'content' in data and data['content'] else None
    return text;
#-----------------------------------------------------------------------------------
def getDataFromRaw(ffile, maxsize=5*1024*1024):
    size = os.path.getsize(ffile)
    if (size > maxsize):
        print(f"File size is too large {size/1024/2014}MB; max is {maxsize}")
        return ffile, ffile
                
    with open(ffile, "rb") as f:
        raw =f.read();
    try:
        data = base64.b64encode(raw).decode('ascii')
    except:
        data = ""
    return (data);

#-----------------------------------------------------------------------------------
def getDataTExtract(ffile):
    text = textract.process(ffile)
    return text
            
    return None;
#-----------------------------------------------------------------------------------
def getData(ffile):
    text = f"ERROR: Unable to read from {ffile}"
    try:
        text = getDataTExtract(ffile)
        return text
    except Exception as e:
        #print(f'Error Textract: {e}')
        pass;
    
    try:
        text = getDataTika(ffile)
        return text
    except Exception as e:
        print(f'Error Tika: {e}')

    return text;
#-----------------------------------------------------------------------------------
def split_paragraph(text, max_paragraph_size = 512):
    documents = [text]
    
    if (len(text) <= max_paragraph_size):
        return documents
    
    paras = [f.strip() for f in re.split(r'[\n\r][\n\r]+', test) if f]

    if ( not paras ):
        return documents;
    
    documents = []
    last_para = ""
    for para in paras:
        para = para.strip()
        if not para:
            continue
        if not last_para:
            last_para = para
            continue;
            
            
        if len(last_para) < max_paragraph_size and (len(para) < 10 or len(re.findall(r"\s+", para)) < 2):
            last_para += "\n"+para
        else:
            documents.append(last_para)
            last_para = para
            
    if ( last_para):
        documents.append(last_para)
    
    return documents
    
#-----------------------------------------------------------------------------------
def inJupyter():
    try:    get_ipython; return True
    except: return False
if __name__ == '__main__':
    if ( not inJupyter()):
        if (len(sys.argv) <= 1):
            print("No arguments? give file name")
            sys.exit(1)
        file = os.path.expanduser(sys.argv[1])
        data = getData(file)   
        print(f'''{data[0:512]}''')
    else: 
        pass

#getData(os.path.expanduser("~/Downloads/test.png"))


Overwriting /opt/utils/getraw.py


In [1]:
%%writefile /opt/utils/filestoes.py
#!/usr/local/bin/python 

import sys, os, datetime, getopt, math,  json, argparse, glob, elasticsearch
from elasticsearch import Elasticsearch, helpers
import base64, pathlib
import getraw

'''
*** NOTE: DO NOT EDIT THIS FILE - THIS iS CREATED FROM: aiservices/notebooks/es/loadtoelastic.ipynb
Indexes all the files into elastic search - currently running on local machine
'''

#-----------------------------------------------------------------------------------
def getfile(file_name, index='', split_paragraphs=False, **kwargs):

    ext = pathlib.Path(file_name).suffix[1:]

    if ( ext in "md meta csv txt".split()):
        with open(file_name, "rb") as f:
            contents =f.read();        
    else:
        contents = getraw.getData(file_name)
        
    if (not type(contents) == str):
        contents = contents.decode('ascii', 'ignore')

    meta ={}
    fmeta= file_name+".meta"
    
    if (os.path.exists(fmeta)):
        with open(fmeta, "rb") as f:
            cmeta =f.read();        
            meta = json.loads(cmeta)
        
    contents = contents.strip()
    if ( not contents ):
        print(f"No data in {file_name}")
        return None;

    #We can use this if storing abspath is dangerous
    #id1 = "".join([f'{i:03}' for i in "/cikiosk/notebooks/sada".encode()])
    #OR
    #id1 = Hash("/cikiosk/notebooks/sada")

    ftype = pathlib.Path(file_name).suffix[1:]
    rfile_name = file_name
    if (ftype == "meta"):
        rfile_name = file_name[:-5]  # Remote ".meta extension"
        
    id1     = os.path.abspath(rfile_name)
    id1     = id1.encode('ascii', 'ignore').decode()    
    size    = os.path.getsize(rfile_name)
    stats   = os.stat( rfile_name )
    docType = pathlib.Path(rfile_name).suffix[1:] or "_doc"
    doc_source = {
        "file_name"  : rfile_name,
        "id"         : id1,
        "full_path"  : id1,
        "public"     : "on",
        "approved"   : 0,
        "comments"   : {'id': 0, "comment": "", "time": "", 'rating': ""},
        "author"     : meta.get("user", ''),
        "index"      : index,
        "editors"    : meta.get("collaborators", ''),
        "type"       : docType,
        "create_time": datetime.datetime.fromtimestamp(stats.st_mtime),
        "modify_time": datetime.datetime.fromtimestamp(stats.st_mtime),
        "content"    : contents,
        'rev'        : 1.0, 
        'votesup'    : 0,
        'votesdown'  : 0,
        "rating"     : 0,
        "score"      : 0
    }
    if (ftype == "meta"):
        d1 =json.loads(contents)
        doc_source.update(d1)
        os.rename(file_name, f'{file_name}.1')

    op = {
        "_index": doc_source['index'],
        #"_type" : docType,
        "_id": id1, # number _id for each iteration
        "_source": doc_source
    }
    
    print(f"File:{id1} {len(contents)} type: {docType} {index}")
    return op
#-----------------------------------------------------------------------------------
def _yield_files():
    ppath = sysargs.path+"/"+sysargs.pattern
    if ( sysargs.recurse ):
        ppath = sysargs.path+"/**/"+sysargs.pattern
        
    #files = glob.glob(ppath)
    files = [f for f in glob.glob(ppath, recursive=sysargs.recurse) if os.path.isfile(f)]
    print (f'Path: {ppath}, R: {sysargs.recurse}, files:({len(files)}): {files[0:5]} ... ')
    return files

def yield_docs():
    files = _yield_files()
    count = 0

    for ffile in files:
        if ( count >= sysargs.number):
            return count;
        
        #If already indexed - dont index it again
        indexed = f'{ffile}.mdd'; 
        if os.path.exists( indexed):
            print(f"-Ignoring {indexed} exists")
            continue;
        
        #ddir = os.path.dirname(ffile)+"/indexed"
        #if (not os.path.exists(ddir)): os.mkdir(ddir)
        if ( sysargs.pattern != "*.meta" and (ffile.endswith("meta") or ffile.endswith("mdd")) ):
            print(f"-Ignoring {ffile} because of no explicit pattern. Call with 't \*.meta'")
            continue;
        
        print(f"+Indexing {ffile}")
        try:
            count += 1
            op= getfile(ffile, sysargs.index, sysargs.paras)
            if (not op):
                continue;
            with open (indexed, "w") as f: f.write("")
            yield op
        except Exception as e:
            print(f"*** ERROR *** {ffile} {e}")
    
#-----------------------------------------------------------------------------------
def deleteAll(es=None):
    if not es:
        es = Elasticsearch([{'host': sysargs.host, 'port': sysargs.port}])
    
    k=[k for k in es.indices.get_alias('*').keys()]
    for i in k:
        print(f'deleteing {i}')
        es.indices.delete(index=i)
    
#-----------------------------------------------------------------------------------
def createSome(es=None):
    if not es:
        es = Elasticsearch([{'host': sysargs.host, 'port': sysargs.port}])
    es.index(index='internal', doc_type = 'internal', id=1, body={
        'rev':1.1,
        'type': 'report',
        'author': 'Boeing Martin',
        'title': 'LM airplane better than boeing airplane',
        'topics': ['airplane', 'engine', 'lockheed', 'boeing', 'pilot'],
        'link': 'linktoactualreport',
        'content': 'we can put actual text from the body of the document here',
        'priority': 1
    })        
#-----------------------------------------------------------------------------------    
def main():
    print(f'''Will start to index files from directory {sysargs.path} - {sysargs.verify} ''')
    
    if (sysargs.verify):
        v = input("is this OK? (Y/n):")
        if ( v.lower() == 'n'):
            print("User aborting ...")
            return 

    es = Elasticsearch([{'host': sysargs.host, 'port': sysargs.port}])
    
    try:
        #_yield_files()
        resp = helpers.bulk( es, yield_docs())
        print ("\nhelpers.bulk() RESPONSE:", resp)
    except Exception as err:
        print("\nhelpers.bulk() ERROR:", err)

#-----------------------------------------------------------------------------------
sysargs=None
desc = '''
This program will index files from a directory into elastic search.
Make sure you install and configure elastic search. When ready to index files, change to that directory 
and run:
\n\n

    filestoes.py -t "*.meta" 

'''
def addargs():
    sysargs = None
    p = argparse.ArgumentParser(f"{os.path.basename(sys.argv[0])}:", description=desc)
    p.add_argument('-H', '--host',   type=str,             required=False, default="localhost", help="host"                 )
    p.add_argument('-p', '--port',   type=int,             required=False, default=9200,        help="ES port number"       )
    p.add_argument('-d', '--path',   type=str,             required=False, default=".",         help="directory of files"   )
    p.add_argument('-i', '--index',  type=str,             required=False, default="aiml",      help="Index Name"           )
    p.add_argument('-n', '--number', type=int,             required=False, default=1024*1024,   help="Max files to index"   )
    p.add_argument('-s', '--skip',   type=int,             required=False, default=0,           help="skip # of files"      )
    p.add_argument('-t', '--pattern',type=str,             required=False, default="*.*",       help="File patterns"        )
    p.add_argument('-f', '--force',  action='store_true',  required=False, default=False,       help="Reindex "             )
    p.add_argument('-a', '--attach', type=bool,            required=False, default=True,        help="As Attachment"        )
    p.add_argument('-x', '--maxsize',type=int,             required=False, default=5*1024*1024, help="Maxsize"              )
    p.add_argument('-P', '--paras'  ,action='store_true',  required=False, default=False,       help="Split text to paras"  )
    p.add_argument('-R', '--recurse',action='store_true',  required=False, default=False,       help="recurse"  )
    p.add_argument('-v', '--verify', action='store_true',  required=False, default=False,       help="Verify"   )
    p.add_argument('-D', '--delete', action='store_true',  required=False, default=False,       help="**DELETE INDICES**"   )
    p.add_argument('-C', '--create', action='store_true',  required=False, default=False,       help="**Create random stuff")
    p.add_argument('files',nargs='*')

    try:
        sysargs=p.parse_args(sys.argv[1:])
    except argparse.ArgumentError as exc:
        print(exc.message )

    return sysargs
#-----------------------------------------------------------------------------------
def inJupyter():
    try:    get_ipython; return True
    except: return False
if __name__ == '__main__':
    if ( not inJupyter()):
        t1 = datetime.datetime.now()
        sysargs = addargs()
        
        if ( sysargs.delete ):
            deleteAll()
        elif ( sysargs.create ):
            createSome()
        else:
            main()
        t2 = datetime.datetime.now()
        print(f"{sys.argv[0]}: All Done in {str(t2-t1)} ***")
    else: 
        pass
    

Overwriting /opt/utils/filestoes.py


In [4]:
#%%writefile /opt/utils/filestoes-OLD.py
#!/usr/local/bin/python 
'''
#-----------------------------------------------------------------------------------
def createPipeline(es, index):
    body = {
      "description" : "Extract attachment information",
      "processors" : [
        {
          "attachment" : {
            "field" : "data"
          }
        }
      ]
    }
    ret = es.index(index='_ingest', doc_type='pipeline', id='attachment', body=body)
    return ret;

#-----------------------------------------------------------------------------------
def IndexFile(es, id, body, index='internal', doc_type="_doc"):
    result2 = es.index( index=index, doc_type=doc_type, pipeline=pipe, id = id1, body=body)
    return result2;

#-----------------------------------------------------------------------------------
def IndexDirFiles1(es, rootdir, index, docType="", force=0, howmany=1024*1024):
    count = 0
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if ( count >= howmany):
                return count;
            ffile= os.path.join(subdir, file)
            
            #We can use this if storing abspath is dangerous
            #id1 = "".join([f'{i:03}' for i in "/cikiosk/notebooks/sada".encode()])
            #OR
            #id1 = Hash("/cikiosk/notebooks/sada")
            
            indexed = f"indexed/{ffile}"
            if ffile.endswith("indexed") or os.path.exists(indexed):
                continue;
                
            id1 = os.path.abspath(ffile)
            id1 = id1.encode('ascii', 'ignore').decode()
                
            try:
                text = getraw.getData(ffile)
                text = text.decode('ascii', 'ignore')
            except Exception as e:
                print(f"{Exception: {ffile} : {e}}")
                continue;
                
            if ( not text ):
                print("No data")
                continue
                
            try:
                size = os.path.getsize(ffile)
                if not sysargs.attach or size > sysargs.maxsize:
                    attach = ""
                else:
                    with open(ffile, "rb") as f:
                        raw =f.read();
                        attach = base64.b64encode(raw).decode('ascii')
                
                body = {'data': attach, 'text': text, 'rev': 1.0, 'votes': 1.0, 'id': id1 }
                
                docType = pathlib.Path(ffile).suffix[1:] or "notype"
                
                print(f"File:{id1} count: {count}/{howmany}, {len(text)} type: {docType} {index}")
                result2 = IndexFile(es, body, index, docType, id1, pipe="attachment")
                count += 1
                
                if ( not os.path.exists("indexed")):
                    print("Creating directory indexed")
                    os.makedir("indexed")
                with open("indexed/"+ffile+".indexed", "w") as f:
                    pass;
                #if (move(os.path.abspath(ffile), os.path.abspath(ffile)+".indexed")):
                #    od.move("")
            except Exception as e:
                print(e)
                return;
    
        if sysargs.recurse:
            for d in dirs:
                if ( d.endswith("indexed")):
                    continue;
                IndexDirFiles(es, os.path.join(subdir, d), index, docType, force, howmany, recurse)
                count += 1
        return count;
            
#-----------------------------------------------------------------------------------
def main1():
    print(f'Will start to index files from directory {sysargs.path} - ')
    v = input("is this OK? (y/N):")
    if ( v.lower() == 'n'):
        print("User aborting ...")
        return 
    es = Elasticsearch([{'host': sysargs.host, 'port': sysargs.port}])
    createPipeline(es, sysargs.index);
    c = IndexDirFiles1(es, sysargs.path, sysargs.index) 
'''

'\n#-----------------------------------------------------------------------------------\ndef createPipeline(es, index):\n    body = {\n      "description" : "Extract attachment information",\n      "processors" : [\n        {\n          "attachment" : {\n            "field" : "data"\n          }\n        }\n      ]\n    }\n    ret = es.index(index=\'_ingest\', doc_type=\'pipeline\', id=\'attachment\', body=body)\n    return ret;\n\n#-----------------------------------------------------------------------------------\ndef IndexFile(es, id, body, index=\'internal\', doc_type="_doc"):\n    result2 = es.index( index=index, doc_type=doc_type, pipeline=pipe, id = id1, body=body)\n    return result2;\n\n#-----------------------------------------------------------------------------------\ndef IndexDirFiles1(es, rootdir, index, docType="", force=0, howmany=1024*1024):\n    count = 0\n    for subdir, dirs, files in os.walk(rootdir):\n        for file in files:\n            if ( count >= howmany):

In [5]:
import pathlib
f="files.csv"
ext=pathlib.Path(f).suffix[1:]

if ( ext in "md meta csv txt".split()):
    print("Bumm bu ...")


Bumm bu ...


In [11]:
import re
f='''
sada
baby, sds sfsf
sdsadsa
'''

n= re.sub('\nbaby.*\n', "\nNew Line\n", f)
print(n)


sada
New Line
sdsadsa

